# Cytoscape Jupyter Widget Demo

Keiichiro ONO (kono at ucsd dot edu)
10/8/2018

### Official Web Site and Code Repository

* https://github.com/idekerlab/cytoscape-jupyter-widget 

### PyPi

* (coming soon...)

![](https://cytoscape.org/images/logo/cy3logoOrange.svg)

## Introduction

This is a demo notebook for Cytoscape Jupyter Widget: graph visualizer for Jupyter Notebook.  It's a very simple graph visualizer using [Cytoscape.js](http://js.cytoscape.org).  You can interactively visualize graphs (networks) in CX or Cytoscape.js format.

## Install Widget

```bash
git clone　https://github.com/idekerlab/cytoscape-jupyter-widget.git
cd cytoscape-jupyter-widget
./build.sh
```

## How to Use the Widget

### 1. Import Widget

In [1]:
from cyjupyter import Cytoscape

### 2. Prepare Data

This widget supports graph data in CX or Cytoscape.js JSON (cyjs).

#### Cytoscape.js JSON

Cytoscape.js JSON is a simple format for graph data.  The following code generates Cytoscape.js data as a standard Python dictionary.

In [2]:
# A Cytoscape.js network with 4 nodes and 5 edges

minimal_cyjs_network = {
    'elements': {
        'nodes': [
            {'data' : {'id': 'node 1'}},
            {'data' : {'id': 'node 2'}},
            {'data' : {'id': 'node 3'}},
            {'data' : {'id': 'node 4'}}
        ],
        'edges': [
            { 'data': {'id': 'edge1','source': 'node 1','target': 'node 2'}},
            { 'data': {'id': 'edge2','source': 'node 1','target': 'node 3'}},
            { 'data': {'id': 'edge3','source': 'node 2','target': 'node 3'}},
            { 'data': {'id': 'edge4','source': 'node 2','target': 'node 4'}},
            { 'data': {'id': 'edge5','source': 'node 3','target': 'node 4'}}
        ]
    }
}

#### CX JSON

CX is a JSON format used for network data transfer in the Cytoscape ecosystem.  You can download any network from [NDEx](http://www.ndexbio.org/) in CX format.  The following example uses NDEx Python client to download the network in CX.  You can change the _uuid_ to try any networks in NDEx.

In [94]:
import ndex2.client as nc

# Unique ID for a network entry in NDEx
uuid0 = '568d095e-1e5f-11e8-b939-0ac135e8bacf'
uuid ='f28356ce-362d-11e5-8ac5-06603eb7f303'

# NDEx public server URL
ndex_url = 'http://public.ndexbio.org/'

# Create an instance of NDEx client
ndex=nc.Ndex2(ndex_url)

# Download the network in CX format
response=ndex.get_network_as_cx_stream(uuid)
print('Response code from NDEx: ', response.status_code)
cx = response.json()

Response code from NDEx:  200


### 3. Render it!

#### Cytoscape.js Networks

By default, the widget is configured to display networks in Cytoscape.js JSON.  If you pass pass it as _*data*_ parameter, it automatically render it using default _Visual Style_ and [cose layout](http://js.cytoscape.org/#layouts/cose).

In [5]:
Cytoscape(data=minimal_cyjs_network)

Cytoscape(data={'elements': {'nodes': [{'data': {'id': 'node 1'}}, {'data': {'id': 'node 2'}}, {'data': {'id':…

#### CX Networks

To display CX networks, you just need to pass the List object (CX is always a list of _aspects_ for more information about CX, please read the [official document](http://www.home.ndexbio.org/data-model/)) **AND SPECIFY FORMAT, cx**.  If the data contains Visual Style and layout, it will be applied automatically.  Otherwise, default ones will be used.

In [95]:
Cytoscape(data=cx, format='cx')

Cytoscape(data=[{'numberVerification': [{'longNumber': 281474976710655}]}, {'metaData': [{'consistencyGroup': …

### Use files exported from Cytoscape
You can easily use files edited in [Cytoscape Desktop](https://cytoscape.org/).

In [108]:
import json

# Network file exported from Cytoscape (in .cyjs format)
with open('hsa00020.cyjs') as f:
    kegg = json.load(f)
# Style file exported from Cytoscape (in Cytoscape.js Style JSON format)
with open('kegg-styles.json') as f2:
    kegg_style = json.load(f2)

# Actual style should be extracted...
style_obj = kegg_style[0]['style']
    
Cytoscape(data=kegg, visual_style=style_obj, layout={'height': '700px'})

Cytoscape(data={'format_version': '1.0', 'generated_by': 'cytoscape-3.6.1', 'target_cytoscapejs_version': '~2.…

## Optional Parameters

You can use most of the basic Cytoscape.js features through optional parameters.

### Cell Height

This widget uses [standard Layout mechanism for Jupyter Widgets](https://minrk-ipywidgets.readthedocs.io/en/latest/examples/Widget%20Styling.html#The-layout-attribute.).  To specify the cell height, you can pass it as a part of _layout_ parameter:

In [7]:
Cytoscape(data=cx, format='cx', layout={'height': '300px'})

Cytoscape(data=[{'numberVerification': [{'longNumber': 281474976710655}]}, {'metaData': [{'name': 'provenanceH…

### Automatic Graph Layout

If there is no **layout_name** parameter, default one will be used.  To specify one of the layout algorithms available in Cytoscape.js, just pass the name:

In [8]:
Cytoscape(data=cx, format='cx', layout_name='grid')

Cytoscape(data=[{'numberVerification': [{'longNumber': 281474976710655}]}, {'metaData': [{'name': 'provenanceH…

### Custom _Visual Style_

This widget fully supports Cytoscape.js compatible styls.  Just pass your style as a dict.  For more information about styling, please visit Cytoscape.js official web site:

* Cytoscape.js [Style](http://js.cytoscape.org/#style)

Also, you can change background color by passing _background_ parameter as string.

In [18]:
# Custom visual style as Python Dict
my_style = [
    {
        'selector': 'node',
        'style': {
            'background-color': '#EFEFEF',
            'label': 'data(name)',
            'width': 10,
            'height': 10,
            'shape': 'diamond',
            'color': '#EEEEEE',
            'font-weight': 400,
            'text-halign': 'right',
            'text-valign': 'bottom',
            'font-size': 12
        }
    },
    {
        'selector': 'edge',
        'style': {
            'width': 1,
            'opacity': 0.4,
            'line-color': 'green',
            'target-arrow-color': 'green',
            'target-arrow-shape': 'triangle'
        }
    }
]

Cytoscape(data=cx, format='cx', layout_name='circle', visual_style=my_style, background='#000000')

Cytoscape(background='#000000', data=[{'numberVerification': [{'longNumber': 281474976710655}]}, {'metaData': …

## Use with Other Tools and Libraries

This widget is a simple network visualizer, but is powerful if you use with other popular libraries.

### Network X

[NetworkX](https://networkx.github.io/) is a widely used library for network analysis.  It has basic visualization features, but you can use advanced visualization feature of Cytoscape.js using this widget.

**NetworkX v2.x series has I/O library for Cytoscape.js objects, and we strongly recommend to use v2.2 or newer.**

In [91]:
# You must install NetworkX before running the following cells!
import networkx as nx
from networkx.readwrite import cytoscape_data
from networkx.algorithms import betweenness_centrality, edge_betweenness_centrality

# Generate a scale-free graph
# G = nx.scale_free_graph(200)
G = nx.waxman_graph(200)

# Calculate PageRank
bc = betweenness_centrality(G)
ebc = edge_betweenness_centrality(G)
nx.set_node_attributes(G, bc, 'betweenness')
nx.set_edge_attributes(G, ebc, 'betweenness')

bc_min = min(bc.values())
bc_max = max(bc.values())

ebc_min = min(ebc.values())
ebc_max = max(ebc.values())

mapper = 'mapData(betweenness,' + str(bc_min) + ',' + str(bc_max) + ', 5, 80)'
op_mapper = 'mapData(betweenness,' + str(bc_min) + ',' + str(bc_max) + ', 0.3, 1)'
color_mapper = 'mapData(betweenness,' + str(bc_min) + ',' + str(bc_max) + ', #EFEFEF, #00897B)'

edge_mapper = 'mapData(betweenness,' + str(ebc_min) + ',' + str(ebc_max) + ', 1, 10)'
edge_op_mapper = 'mapData(betweenness,' + str(ebc_min) + ',' + str(ebc_max) + ', 0.1, 1)'


my_style2 = [
    {
        'selector': 'node',
        'style': {
            'background-color': color_mapper,
            'width': mapper,
            'height': mapper,
            'background-opacity': op_mapper
        }
    },
    {
        'selector': 'edge',
        'style': {
            'width': edge_mapper,
            'opacity': edge_op_mapper,
            'line-color': '#333333'        
        }
    }
]

# Layout using NetworkX
pos = nx.kamada_kawai_layout(G)
cyjs2 = cytoscape_data(G)

nodes = cyjs2['elements']['nodes']

# Assign (X, Y) positions
SCALE_FACTOR = 600
for node in nodes:
    node_id = node['data']['id']
    node['position'] = {
        'x': pos[int(node_id)][0] * SCALE_FACTOR,
        'y': pos[int(node_id)][1] * SCALE_FACTOR
    }

Cytoscape(data=cyjs2,  layout_name='preset', visual_style=my_style2, layout={'height': '900px'})

Cytoscape(data={'data': [], 'directed': False, 'multigraph': False, 'elements': {'nodes': [{'data': {'pos': (0…